In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../../")

from experiment import Exp

from dataloaders import data_set,data_dict
import torch
import yaml
import os

# 参数设置

# 训练参数 
除了路径 其他不要变

In [3]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()   
# TODO change the path as relative path
args.to_save_path     = r"C:\Users\Murat\Desktop\Bachelor\models\ISWC2022LearnableFilter\Run_logs"
args.freq_save_path   = r"C:\Users\Murat\Desktop\Bachelor\models\ISWC2022LearnableFilter\Freq_data"
args.window_save_path = r"C:\Users\Murat\Desktop\Bachelor\models\ISWC2022LearnableFilter\Sliding_window"
args.root_path        = r"C:\Users\Murat\Desktop\Bachelor\datasets"


args.drop_transition  = False
args.datanorm_type    = "standardization" # None ,"standardization", "minmax"


args.batch_size       = 256                                                     
args.shuffle          = True
args.drop_last        = False
args.train_vali_quote = 0.90                                           


# training setting 
args.train_epochs            = 150

args.learning_rate           = 0.001  
args.learning_rate_patience  = 5
args.learning_rate_factor    = 0.1


args.early_stop_patience     = 15

args.use_gpu                 = True if torch.cuda.is_available() else False
args.gpu                     = 0
args.use_multi_gpu           = False

args.optimizer               = "Adam"
args.criterion               = "CrossEntropy"

## 数据参数



In [4]:
args.seed                             = 1


args.data_name                        =  "hapt"

args.wavelet_filtering                = False
args.wavelet_filtering_regularization = False
args.wavelet_filtering_finetuning     = False
args.wavelet_filtering_finetuning_percent = 0.3

args.regulatization_tradeoff          = 0.0001
args.number_wavelet_filtering         = 10


args.difference       = True 
args.filtering        = True
args.magnitude        = True
args.weighted_sampler = False




args.pos_select       = None
args.sensor_select    = None


args.representation_type = "time"
args.exp_mode            = "LOCV"

config_file = open('../../configs/data.yaml', mode='r')
data_config = yaml.load(config_file, Loader=yaml.FullLoader)
config = data_config[args.data_name]

args.root_path       = os.path.join(args.root_path,config["filename"])
args.sampling_freq   = config["sampling_freq"]
args.num_classes     =  config["num_classes"]
window_seconds       = config["window_seconds"]
args.windowsize      =   int(window_seconds * args.sampling_freq) 
args.input_length    =  args.windowsize
# input information
args.c_in            = config["num_channels"] * 3 if args.difference else config["num_channels"]
args.p = {
            'jitter': 1.0,
            'exponential_smoothing': 1.0,
            'moving_average': 1.0,
            'magnitude_scaling': 1.0,
            'magnitude_warp': 1.0,
            'magnitude_shift': 1.0,
            'time_warp': 1.0,
            'window_warp': 1.0,
            'window_slice': 1.0,
            'random_sampling': 1.0,
            'slope_adding': 1.0,
        }
args.mixup_lambda = 0.5
args.max_randaug_cnt = 3

if args.wavelet_filtering :
    
    if args.windowsize%2==1:
        N_ds = int(torch.log2(torch.tensor(args.windowsize-1)).floor()) - 2
    else:
        N_ds = int(torch.log2(torch.tensor(args.windowsize)).floor()) - 2

    args.f_in            =  args.number_wavelet_filtering*N_ds+1
else:
    args.f_in            =  1



## 模型参数

In [5]:
args.filter_scaling_factor = 0.25
args.model_type            = "deepconvlstm"

# 实验

In [6]:
exp = Exp(args)

Use GPU: cuda:0
Build the DeepConvLSTM model!
Done!
Parameter : 45596
Set the seed as :  1


In [7]:
exp.train()

 ----------------------- load all the data -------------------
Channel Augmentation : Differencing
Channel Augmentation : Acc Gyro Filtering
Channel Augmentation : Magnitute Calculating for acc and Gyro
-----------------------Sliding file are generated -----------------------
-----------------------Sliding file are generated -----------------------
================ LOCV Mode ====================
================ 30 CV ======================
================ the 0 th CV Experiment ================ 
Leave one Out Experiment : The 1 Part as the test
[-] Target sampling weights:  [0.00061996 0.00062775 0.0006854  0.00057241 0.00052521 0.00053821
 0.00689655 0.01010101 0.00625    0.00675676 0.00512821 0.00680272]
Train data number :  11068
The number of classes is :  12
The input_length  is :  128
The channel_in is :  18
Validation data number :  1230
Test data number :  2326
================ Build the model ================ 
Build the DeepConvLSTM model!
(18, 128)
12
(18, 128)
127
(18, 128

RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Float'